In [163]:
dataDir= "D:/Autonomous Vehicle/Code1/location-2"

In [ ]:
#exec(open('init_notebook.py').read())
exec(open('D:/Autonomous Vehicle/Code2/behavior-hypotheses/src/notebooks/init_notebook.py').read())

In [165]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import numpy as np
from scipy.interpolate import interp1d

from tabulate import tabulate
from tti_dataset_tools.TrajectoryTransformer import TrajectoryTransformer
from tti_dataset_tools.TrajectoryVisualizer import TrajectoryVisualizer
from tti_dataset_tools.TrajectoryCleaner import TrajectoryCleaner
from behavior_tools.TrajectoryADECalculator import TrajectoryADECalculator
from behavior_tools.Sampler import Sampler
from tti_dataset_tools.ColMapper import ColMapper

from behavior_tools.AngularDisplacementCalculator import AngularDisplacementCalculator

In [166]:
sceneId=210
date="2023-05-04"
fps=10
minYDisplacement=5
maxXDisplacement=6
pedDf=pd.read_csv(os.path.join(dataDir,f"{date}-fps-{fps}-scene-{sceneId}-pedestrians.csv"))
otherDf=pd.read_csv(os.path.join(dataDir,f"{date}-fps-{fps}-scene-{sceneId}-others.csv"))
tracksMeta=pd.read_csv(os.path.join(dataDir,f"{date}-fps-{fps}-scene-{sceneId}-meta.csv"))

In [167]:
visualizer=TrajectoryVisualizer()
adeCalculator=TrajectoryADECalculator()
colMapper= ColMapper(idCol='uniqueTrackId',
                     xCol='sceneX',
                     yCol='sceneY',
                     xVelCol='sceneXVelocity',
                     yVelCol='sceneYVelocity',
                     speedCol='speed',
                     fps=fps
                     )
transformer=TrajectoryTransformer(colMapper)
cleaner=TrajectoryCleaner(colMapper=colMapper,
                          minSpeed=0.0,
                          maxSpeed=3.5,
                          minYDisplacement=minYDisplacement,
                          maxXDisplacement=maxXDisplacement)
transformer.deriveSpeed(pedDf)
transformer.deriveDisplacements(pedDf)

In [ ]:
cleanPedDf=cleaner.cleanByYDisplacement(pedDf)
#visualizer.show(pedDf,idCol='uniqueTrackId',xCol='sceneX',yCol='sceneY')
#visualizer.show(cleanPedDf,idCol='uniqueTrackId',xCol='sceneX',yCol='sceneY')
#print(cleanPedDf)
allPedIds=list(cleanPedDf["uniqueTrackId"].unique())
#print(allPedIds)->Gives the unique ids
transformer.translateAllToLocalSource(cleanPedDf)# Creates two new columns localX,localY which gives the values of points with resopect to one single origin
#print(cleanPedDf)
#visualizer.show(cleanPedDf,idCol="uniqueTrackId",xCol="localX",yCol="localY")

southIds=[]
for pedId in allPedIds:
        trackDf=cleanPedDf[cleanPedDf['uniqueTrackId']==pedId]
        trackMeta=tracksMeta[tracksMeta['uniqueTrackId']==pedId].iloc[0]
        
        if trackMeta['verticalDirection']=="SOUTH":
                southIds.append(pedId)
                X,Y=transformer.rotate180(trackDf,'localX','localY')
                cleanPedDf.loc[cleanPedDf['uniqueTrackId']==pedId,'localX']=X
                cleanPedDf.loc[cleanPedDf['uniqueTrackId']==pedId,'localY']=Y
#visualizer.show(cleanPedDf,idCol='uniqueTrackId',xCol='localX',yCol='localY')
print(allPedIds)

In [169]:
## collect localXe and localYe
trackIds = cleanPedDf["uniqueTrackId"].unique()
localXe=[]
localYe=[]

for trackId in trackIds:
    trackDf = cleanPedDf[cleanPedDf["uniqueTrackId"] == trackId]
    
    #collect and append the localX and localY values in a list
    localXe.append(trackDf["localX"].tolist())
    localYe.append(trackDf["localY"].tolist())


In [170]:
def calculate_threshold(points) :
        i=0
        sum=0
        size=len(points)
        for i in range(size-1) :
                sum=sum+abs(points[i][0]-points[i+1][0])
        return abs(sum)/size

In [171]:
def rdp(points, epsilon):
    """
    Reduce the number of points in a curve using the Ramer-Douglas-Peucker algorithm.
    Parameters:
    - points: Nx2 array of x and y coordinates.
    - epsilon: Tolerance parameter.
    Returns:
    - Reduced set of points.
    """
    # Find the point with the maximum distance from the line between start and end
    start, end = points[0], points[-1]
    line_vec = end - start
    line_vec_norm = line_vec / np.hypot(*line_vec)
    distances = np.abs(np.cross(points - start, line_vec_norm))
    
    
    max_distance = np.max(distances)
    index = np.argmax(distances)
    
    
    # If the max distance is greater than epsilon, recursively simplify
    if max_distance > epsilon:
        left = rdp(points[:index+1], epsilon)
        right = rdp(points[index:], epsilon)
        return np.vstack((left[:-1], right))
    else:
        return np.array([start, end])

In [172]:
def rdp_limited(points, max_nodes, initial_epsilon, step):
    epsilon = initial_epsilon
    while True:
        reduced_points = rdp(points, epsilon)
        if len(reduced_points) <= max_nodes:
            return reduced_points
        else:
            epsilon += step  # Simplify more by increasing epsilon
        

In [ ]:

## main algo

for i in range(100,110):
    dataX = (np.array(localXe[i]))
    dataY = (np.array(localYe[i]))
    points= np.column_stack((dataX, dataY))
    max_nodes=8
    epsilon = calculate_threshold(points)/5
    step=epsilon/2
    reduced_points = rdp_limited(points,max_nodes, epsilon,step)
    x_nodes = reduced_points[:, 0]
    y_nodes = reduced_points[:, 1]

    plt.figure(figsize=(20,10))
    plt.title(f"Track {allPedIds[i]}")
    plt.plot(dataX, dataY, label='Original Data', color='blue')
    plt.plot(x_nodes, y_nodes, label='Refined Linear Spline', color='orange')
    plt.scatter(x_nodes, y_nodes, color='red', label='Spline Nodes')
    plt.legend()

plt.show()